In [ ]:
# 1.值范围 2.相似线段形态 3.highlight落在区间内 4.方差大

In [1]:
import json
import uuid
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}
for index in range(12):
    item = {}
    item['role_path'] = 'root.' + 'branch' + str(branch[index]) + '.search_branch' + str(branch[index])
    item['identifier'] = 'search_branch' + str(branch[index])
    item['desc'] = 'Branch ' + str(branch[index]) + ' common use KeyValue and Mark'
    item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+').*(txAtt|Pma|linearization fault)'
    item['exp_extract'] = [
                        "{}[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "{}[{timestamp}] {}"
                        ]
    item['exp_mark'] = [{"abbr":"LF","exp":"linearization fault","color":"#f00000"}]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    if index == 0:
        item['is_active'] = True
    config['search'].append(item)

keys = ['txAtt', 'torTemperature', 'avgIMpa0', 'LF']
for search_config in config['search']:
    item = {}
    item['role_path'] = search_config['role_path'].replace('search', 'chart')
    item['identifier'] = search_config['identifier'].replace('search', 'chart')
    item['desc'] = item['identifier'] + ' all KeyValue and Mark'
    item['check'] = False
    item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
    for search_config2 in config['search']:
        search_atom = {}
        search_atom['namespace'] = search_config2['identifier']
        search_atom['name'] = search_config2['identifier']
        search_atom['check'] = False
        search_atom['children'] = []
        for key in keys:
            if search_config['identifier'] == search_config2['identifier']:
                search_atom['children'].append({'name': key, 'check': True})
            else:
                search_atom['children'].append({'name': key, 'check': False})
        item['key_value_tree']['children'].append(search_atom)
    config['chart'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'insight_branch' + str(branch[index])
#     item['desc'] = 'Branch Insight' + str(branch[index])
#     item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+')'
#     item['exp_extract'] = "{}[{timestamp}]{}, msg = {msg}"
#     item['exp_mark'] = {"name":"LF","exp":"linearization fault","color":"#f00000"}
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['insight'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'statistic_branch' + str(branch[index])
#     item['desc'] = 'Branch ' + str(branch[index]) + ' statistic txAtt'
#     item['code'] = "self.result = self.text_file_model.data['" + 'search_branch' + str(branch[index]) + "'].res_key_value['txAtt']['global_index']"
#     config['statistic'].append(item)
    
json_object = json.dumps(config)
with open("E:\\projects\\ericsson_flow\\new_files\\config.ecfg", "w") as outfile:
    outfile.write(json_object)

In [51]:
import re
import time
from textblob import TextBlob

def self_clean_special_symbols(text, symbol):
    text = re.sub(r'0x[\da-fA-F]+', symbol, text)
    for ch in [':', '/','{','}','[',']','(',')','#','+','!',';',',','"','\'','@','`','$','^','&','|','-','.','=','\n']:
        if ch in text:
            text = text.replace(ch,symbol)
            
    text = re.sub(r'\d+', '', text)
    text = re.sub(symbol+"+", symbol, text).strip()
    return text

# string = '[14:56:47.148782281] (+0.000774132) air6419_mongoose com_ericsson_trithread:INFO: { cpu_id = 0 }, { process = "radioTrDcServer", fileAndLine = "trDcPhantomProc.cc:52", msg = "radioTrDcServer recevied msg: 0x170607e, itc_sender: 0x3300002" }'
string = 'DU1_TE_LOG:[2023-02-23 10:24:32.350952212] (+0.000003940) du1 com_ericsson_rc_nwif_f1:MSG_SENT: { cpu_id = 3 }, { procname = "RcNwifF1H1" }, { tid = 12647, msg_id = 18, prot_id = 16786, prot_name = "RcHandlerCtrl", prot_rev = 1, prot_ver = 1214550678, _msg_data_length = 223, msg_data = [ [0] = 0, [1] = 1, [2] = 0, [3] = 18, [4] = 10, [5] = 211, [6] = 1, [7] = 8, [8] = 1, [9] = 18, [10] = 14, [11] = 8, [12] = 224, [13] = 128, [14] = 196, [15] = 5, [16] = 18, [17] = 7, [18] = 8, [19] = 204, [20] = 3, [21] = 16, [22] = 99, [23] = 24, [24] = 2, [25] = 26, [26] = 94, [27] = 10, [28] = 14, [29] = 8, [30] = 224, [31] = 128, [32] = 196, [33] = 5, [34] = 18, [35] = 7, [36] = 8, [37] = 204, [38] = 3, [39] = 16, [40] = 99, [41] = 24, [42] = 2, [43] = 21, [44] = 1, [45] = 0, [46] = 0, [47] = 0, [48] = 26, [49] = 1, [50] = 49, [51] = 42, [52] = 11, [53] = 10, [54] = 7, [55] = 8, [56] = 204, [57] = 3, [58] = 16, [59] = 99, [60] = 24, [61] = 2, [62] = 18, [63] = 0, [64] = 50, [65] = 42, [66] = 10, [67] = 40, [68] = 10, [69] = 12, [70] = 13, [71] = 216, [72] = 192, [73] = 7, [74] = 0, [75] = 26, [76] = 5, [77] = 13, [78] = 7, [79] = 0, [80] = 0, [81] = 0, [82] = 18, [83] = 12, [84] = 13, [85] = 152, [86] = 30, [87] = 8, [88] = 0, [89] = 26, [90] = 5, [91] = 13, [92] = 7, [93] = 0, [94] = 0, [95] = 0, [96] = 26, [97] = 4, [98] = 8, [99] = 0, [100] = 16, [101] = 20, [102] = 34, [103] = 4, [104] = 8, [105] = 0, [106] = 16, [107] = 20, [108] = 58, [109] = 7, [110] = 16, [111] = 17, [112] = 4, [113] = 101, [114] = 65, [115] = 0, [116] = 0, [117] = 72, [118] = 0, [119] = 96, [120] = 1, [121] = 34, [122] = 89, [123] = 10, [124] = 4, [125] = 126, [126] = 99, [127] = 148, [128] = 0, [129] = 18, [130] = 81, [131] = 100, [132] = 128, [133] = 0, [134] = 2, [135] = 10, [136] = 48, [137] = 38, [138] = 64, [139] = 0, [140] = 0, [141] = 64, [142] = 2, [143] = 196, [144] = 1, [145] = 130, [146] = 160, [147] = 128, [148] = 96, [149] = 22, [150] = 0, [151] = 0, [152] = 9, [153] = 246, [154] = 125, [155] = 174, [156] = 16, [157] = 249, [158] = 184, [159] = 32, [160] = 16, [161] = 0, [162] = 0, [163] = 8, [164] = 64, [165] = 0, [166] = 32, [167] = 131, [168] = 66, [169] = 1, [170] = 64, [171] = 97, [172] = 0, [173] = 25, [174] = 194, [175] = 1, [176] = 135, [177] = 192, [178] = 216, [179] = 0, [180] = 0, [181] = 9, [182] = 253, [183] = 92, [184] = 251, [185] = 92, [186] = 44, [187] = 1, [188] = 0, [189] = 2, [190] = 43, [191] = 141, [192] = 67, [193] = 217, [194] = 144, [195] = 2, [196] = 94, [197] = 48, [198] = 134, [199] = 226, [200] = 13, [201] = 247, [202] = 171, [203] = 226, [204] = 0, [205] = 181, [206] = 117, [207] = 0, [208] = 154, [209] = 43, [210] = 220, [211] = 134, [212] = 42, [213] = 2, [214] = 8, [215] = 1, [216] = 48, [217] = 2, [218] = 72, [219] = 0, [220] = 160, [221] = 1, [222] = 0 ] }'
# string = self_clean_special_symbols(string, ' ')
time1 = time.time()

for _ in range(1000):
    string = self_clean_special_symbols(string, ' ')

print(time.time() - time1, string)

0.009846210479736328 DU_TE_LOG du com_ericsson_rc_nwif_f MSG_SENT cpu_id procname RcNwifFH tid msg_id prot_id prot_name RcHandlerCtrl prot_rev prot_ver _msg_data_length msg_data


In [57]:
import json
import uuid
import re

def add_search_config(parent, express, processes):
    config = []
    
    item = {}
    item['identifier'] = f'{parent}_ErrorFaultABN'
    identifier = item['identifier']
    item['role_path'] = f'root.{parent}.{identifier}'

    item['desc'] = parent + ' Error Fault ABN'
    item['exp_search'] = parent + '.*(error| fault|ABN)'
    item['exp_extract'] = [
                            parent + express
                        ]
    item['exp_mark'] = [
                            {"abbr":"ER","exp":"error","color":"#f00000"},
                            {"abbr":"FA","exp":" fault","color":"#f00000"},
                            {"abbr":"ABN","exp":"ABN","color":"#ffff00"}
                        ]
    item['is_active'] = True
    item['is_case_sensitive'] = False
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config.append(item)

    for index, process in enumerate(processes):
        item = {}
        name = process.replace(',','_')
        item['identifier'] = f'{parent}_{name}'
        identifier = item['identifier']
        item['role_path'] = f'root.{parent}.{identifier}'
        item['desc'] =  parent + ' ' + process + ' common use KeyValue and Mark'
        item['exp_search'] = f'{parent}.*{process}'
        item['exp_extract'] = [
                                parent + express
                            ]
        item['exp_mark'] = []
        item['is_active'] = False
        item['is_case_sensitive'] = True
        item['forward_rows'] = 0
        item['backward_rows'] = 0
        config.append(item)
    return config

config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\DOT TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(' \((.*?)\): ', line)
        if len(res) > 0:
            processes.append(res[0]) 
processes = list(set(processes))
processes.sort()
config['search'].extend(add_search_config('DOT_TE_LOG', ":{timestamp} ({}", processes))

# processes = []
# with open("D:\\projects\\ericsson_flow\\ErrorLog\\DU1 TE LOG.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall('procname = \"(.*?)\"', line)
#         if len(res) > 0:
#             processes.append(res[0])
# processes = list(set(processes))
# processes.sort()
# config['search'].extend(add_search_config('DU1_TE_LOG', ":[{timestamp}]{}", processes))
# config['search'].extend(add_search_config('DU2_TE_LOG', ":[{timestamp}]{}", processes))

# processes = []
# with open("D:\\projects\\ericsson_flow\\ErrorLog\\RU1 TE LOG.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall(', { \"(.*?)\"', line)
#         if len(res) > 0:
#             processes.append(res[0])
# processes = list(set(processes))
# processes.sort()
# config['search'].extend(add_search_config('RU1_TE_LOG', '{}[{timestamp}]{}', processes))
# config['search'].extend(add_search_config('RU2_TE_LOG', '{}[{timestamp}]{}', processes))

json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\DotConfigSimple.ecfg", "w") as outfile:
    outfile.write(json_object)

In [49]:
import re
from parse import parse

str1 = 'RU2_TE_LOG:BXP_2: [2023-02-23 10:31:25.766459124] com_ericsson_trithread:INFO: { 0 }, { "TxBranchCtrlC", "logManager.cc:37", "Branch does not support different power level (txChangeCycleStatePrepare.cc:517)" }'
# str2 = 'BXP_3: [2022-12-10 15:55:26.739019220] (+0.000025340) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "TxBranchCtrlB", fileAndLine = "txChangeCycleHelper.cc:264", msg = "Txl branch J restart due to txL linearization fault!" }'
# str1 = "BXP_2: [221120 164014] 27: PA measured values for driver name: DpaVddSv:7; value: 26992; branch Id: 7"
# exp = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# exp = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
exp_extract = '{}[{timestamp}]{}'
r_extract = parse(exp_extract, str1)
print(r_extract)

<Result ('RU2_TE_LOG:BXP_2: ', ' com_ericsson_trithread:INFO: { 0 }, { "TxBranchCtrlC", "logManager.cc:37", "Branch does not support different power level (txChangeCycleStatePrepare.cc:517)" }') {'timestamp': '2023-02-23 10:31:25.766459124'}>


In [41]:
import datetime

datetime.datetime.strptime('2023-02-23 10:31:26.641087', '%Y-%m-%d %H:%M:%S.%f')

datetime.datetime(2023, 2, 23, 10, 31, 26, 641087)

In [48]:
with open("D:\\projects\\ericsson_flow\\ErrorLog\\RU1 TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(', { \"(.*?)\"', line)
        if len(res) > 0:
            processes.append(res[0]) 
processes = list(set(processes))
processes.sort()
processes

['EquipCtrl',
 'PaSrvA',
 'PaSrvB',
 'PaSrvC',
 'PaSrvD',
 'PaSrvE',
 'PaSrvF',
 'PaSrvG',
 'PaSrvH',
 'RcNwifF1H1',
 'RcNwifF1M',
 'RcNwifF1Sctp',
 'RcNwifX2M',
 'RcNwifXnH1',
 'RcNwifXnM',
 'RcPaging',
 'RcTaNr',
 'RcTcNr',
 'RdClient0',
 'RdClient1',
 'RdClient2',
 'RdClient7',
 'RdEqpFaultCtrl',
 'RpUeAH4',
 'RpUeConfig2',
 'RpUeH5',
 'RpUeH6',
 'RpUeH7',
 'RpUeH8',
 'RpUeMS',
 'RpUeMon3',
 'RpcNwifF1H1',
 'RpcNwifF1M',
 'RpcNwifF1Sctp',
 'RpcPaging',
 'RxBranchCtrlA',
 'RxBranchCtrlB',
 'RxBranchCtrlC',
 'RxBranchCtrlD',
 'RxBranchCtrlE',
 'RxBranchCtrlF',
 'RxBranchCtrlG',
 'RxBranchCtrlH',
 'RxLoService',
 'TxBranchCtrlA',
 'TxBranchCtrlB',
 'TxBranchCtrlC',
 'TxBranchCtrlD',
 'TxBranchCtrlE',
 'TxBranchCtrlF',
 'TxBranchCtrlG',
 'TxBranchCtrlH',
 'TxCoordinationSrvABCDEFGH',
 'TxLoSrvABCDEFGH',
 'TxTimingPhaseCtrlA',
 'TxTimingPhaseCtrlB',
 'TxTimingPhaseCtrlC',
 'TxTimingPhaseCtrlD',
 'TxTimingPhaseCtrlE',
 'TxTimingPhaseCtrlF',
 'TxTimingPhaseCtrlG',
 'TxTimingPhaseCtrlH',
 '

In [5]:
import re

# processes = []
# with open("E:\\projects\\ericsson_flow\\new_files\\ru_lock_unlock_dpd_hw_fault_simple.log", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall("process = \"(.*?)\",", line)
#         if len(res) > 0:
#             if ('BranchCtrl' not in res[0]) & ('PhaseCtrl' not in res[0]):
#                 processes.append(res[0])
            
# processes = list(set(processes))
# processes.sort()
processes = ['AntCalGroupMgr',
             'AntCalHandler',
             'AntCalLogManagerProc',
             'BSAas',
             'CompensationManagerAas',
             'EquipCtrl',
             'WorkerTaskAas',
             'antCalCtrlAas_requestProcessor',
             'cfhe',
             'cmd_proc',
             'faultManagerProc',
             'hwLogWriteProc',
             'lteNrTddSwitchSrvRadon1x0',
             'lteNrTddSwitchSrvRadon1x1',
             'lteNrTddSwitchSrvRadon1x2',
             'lteNrTddSwitchSrvRadon1x3',
             'radioTrDcServer',
             'timeOutSrv',
             'tmoSchedulerEqp',
             'tmoSchedulerTx'
             # 'trDcProc',
             # 'txlProcBranch0',
             # 'txlProcBranch10'
            ]

In [6]:
import json
import uuid

processes = processes
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

item = {}
item['role_path'] = 'root.ErrorFaultABN'
item['identifier'] = 'ErrorFaultABN'
item['desc'] = 'Error Fault ABN'
item['exp_search'] = 'error| fault|ABN'
item['exp_extract'] = [
                    "[{timestamp}] {}" 
                    ]
item['exp_mark'] = [
                        {"abbr":"ER","exp":"error","color":"#f00000"},
                        {"abbr":"LF","exp":"linearization fault","color":"#f00000"},
                        {"abbr":"ABN","exp":"ABN:","color":"#ffff00"}
                    ]
item['is_active'] = True
item['is_case_sensitive'] = True
item['forward_rows'] = 0
item['backward_rows'] = 0
config['search'].append(item)

for index, process in enumerate(processes):
    item = {}
    item['role_path'] = 'root.' + process
    item['identifier'] = process
    item['desc'] =  process + ' common use KeyValue and Mark'
    item['exp_search'] = f'process = "{process}"'
    item['exp_extract'] = [
                        # "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "[{timestamp}] {}" 
                        ]
    # item['exp_mark'] = [
    #                         {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#e00000"},
    #                         {"abbr":"ER","exp":"EVENT_RELEASE","color":"#d00000"},
    #                         {"abbr":"ES","exp":"EVENT_SETUP","color":"#c00000"},
    #                         {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#b00000"},
    #                         {"abbr":"IND","exp":"IND_ACTIVATE","color":"#a00000"}
    #                     ]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config['search'].append(item)

tmp_search = []
processes = range(2)
for index in processes:
    item = {}
    branch_num = str(index)
    item['role_path'] = 'root.' + 'branch' + branch_num + '.search_branch' + branch_num
    item['identifier'] = 'search_branch' + branch_num
    item['desc'] = 'Branch ' + branch_num + ' common use KeyValue and Mark'
    item['exp_search'] = f'(process = "txlProcBranch'+branch_num+'"|process = "TxBranchCtrl'+branch_num+'")'
    item['exp_extract'] = [
                        "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "[{timestamp}] {}" 
                        ]
    item['exp_mark'] = [
                            {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#759aa0"},
                            {"abbr":"ER","exp":"EVENT_RELEASE","color":"#e69d87"},
                            {"abbr":"ES","exp":"EVENT_SETUP","color":"#8dc1a9"},
                            {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#ea7e53"},
                            {"abbr":"IND","exp":"IND_ACTIVATE","color":"#eedd78"}
                        ]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config['search'].append(item)
    tmp_search.append(item)
        
keys = ['txAtt', 'avgIMpa0', 'ED', 'ER', 'ES', 'EA', 'IND']
for search_config in tmp_search:
    item = {}
    item['role_path'] = search_config['role_path'].replace('search', 'chart')
    item['identifier'] = search_config['identifier'].replace('search', 'chart')
    item['desc'] = item['identifier'] + ' all KeyValue and Mark'
    item['check'] = False
    item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
    for search_config2 in tmp_search:
        search_atom = {}
        search_atom['namespace'] = search_config2['identifier']
        search_atom['name'] = search_config2['identifier']
        search_atom['check'] = False
        search_atom['children'] = []
        for key in keys:
            if search_config['identifier'] == search_config2['identifier']:
                search_atom['children'].append({'name': key, 'check': True})
            else:
                search_atom['children'].append({'name': key, 'check': False})
        item['key_value_tree']['children'].append(search_atom)
    config['chart'].append(item)
    
json_object = json.dumps(config)
with open("E:\\projects\\ericsson_flow\\new_files\\6419ConfigSimple.ecfg", "w") as outfile:
    outfile.write(json_object)

In [29]:
import re

text = "I like oranges."
pattern = "^(?!.*jesd|.*dfe|.*radiosw|.*rProxyMedian.cc).*" (txlProcBranch0|TxBranchCtrl0).*(event|IND).*328236

re.findall(pattern, text)

['ranges.', '']

In [ ]:
import json
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

item1 = {}
item1['name'] = 'PackageName'
item1['desc'] = 'Elog package restart name'
item1['exp_search'] = '_R13C193'
item1['exp_extract'] = [
                    "[{tmp1}] {}LMC ID: {packname}\n[{tmp2}] {}\n[{tmp3}] {}\n[{tmp4}] {}\n[{tmp5}] {}\n[{timestamp}] {}"
                    ]
item1['exp_mark'] = []
item1['is_case_sensitive'] = True
item1['forward_rows'] = 0
item1['backward_rows'] = 5
config['search'].append(item1)

item2 = {}
item2['name'] = 'ErrorName'
item2['desc'] = 'ABN: raiseFaultAndRollback'
item2['exp_search'] = 'ABN: raiseFaultAndRollback'
item2['exp_extract'] = [
                    "[{timestamp}] {}ABN: {errorname} {}"
                    ]
item2['exp_mark'] = []
item2['is_case_sensitive'] = True
item2['forward_rows'] = 0
item2['backward_rows'] = 0
config['search'].append(item2)

json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\cooper_config.txt", "w") as outfile:
    outfile.write(json_object)

In [18]:
import re
from parse import parse

str1 = 'DOT_TE_LOG:2023-02-23 10:32:14:027 (srv,info): clgc errorStatus: 13581 '
# str2 = 'BXP_3: [2022-12-10 15:55:26.739019220] (+0.000025340) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "TxBranchCtrlB", fileAndLine = "txChangeCycleHelper.cc:264", msg = "Txl branch J restart due to txL linearization fault!" }'
# str1 = "BXP_2: [221120 164014] 27: PA measured values for driver name: DpaVddSv:7; value: 26992; branch Id: 7"
# exp = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# exp = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
exp_extract = 'DOT_TE_LOG:{timestamp} ({}'
r_extract = parse(exp_extract, str1)
print(r_extract)

<Result ('srv,info): clgc errorStatus: 13581 ',) {'timestamp': '2023-02-23 10:32:14:027'}>


In [ ]:
from parse import parse
# str1 = [
#     "221211-00:51:48 10.69.81.43 22.0n MSRBS_NODE_MODEL_22.Q2_566.28125.116_3317 stopfile=/tmp/21234\n",
#     "coli>/fruacc/lhsh BXP_5 ts r\n",
#     "Board                     : 45.2 C\n",
#     "DcPaVdd:0                 : 57.3 C\n",
#     "DcPaVdd:1                 : 57.8 C\n",
#     "DcPaVdd:2                 : 50.1 C\n",
#     "DcPaVdd:3                 : 50.6 C\n",
#     "DcTrxVcc                  : 48.3 C\n"
# ]

st1 = ['221211-00:50:33 10.69.81.43 22.0n MSRBS_NODE_MODEL_22.Q2_566.28125.116_3317 stopfile=/tmp/21234\n', 'coli>/fruacc/lhsh BXP_5 ts r\n', 'Board                     : 33.2 C\n', 'DcPaVdd:0                 : 43.7 C\n', 'DcPaVdd:1                 : 44.3 C\n', 'DcPaVdd:2                 : 37.4 C\n', 'DcPaVdd:3                 : 37.7 C\n', 'DcTrxVcc                  : 48.9 C\n']
str2 = '\n'.join(str1)
r = parse("{timestamp} {}DcPaVdd:0                 : {temp:f} C{}", str2)
r

In [ ]:
import re
import json
from text_analysis import TextAnalysisModel

text_analysis_model = await TextAnalysisModel('parallel', mode = 'test')

file_path = ['D:\\projects\\ericsson_flow\\ELOG_READ_FULL_ENM3\\DL_BAYAHBARATCISIIH_GH.log']
config_path = 'D:\\Projects\\ericsson_flow\\new_files\\cooper_config1.txt'

await text_analysis_model.file_container_model.on_new_file('', file_path)
await text_analysis_model.file_container_model.on_load_config('', config_path)

for key in text_analysis_model.data.keys():
    if 'Name' in key:
        print(key, text_analysis_model.data[key].res_key_value)
    

In [ ]:
import re
import json
from text_analysis import TextAnalysisModel

text_analysis_model = await TextAnalysisModel('parallel', mode = 'test')

dir_path = 'D:\\projects\\ericsson_flow\\batch_test'
config = 'D:\\projects\\ericsson_flow\\new_files\\config2.txt'

await text_analysis_model.file_container_model.batch_insight_model.new(dir_path, config)

In [ ]:
text_analysis_model.file_container_model.batch_insight_model.result.loc[1, 'resOutlier']

In [ ]:
text_analysis_model.file_container_model.batch_insight_model.samples

In [ ]:
tmp = text_analysis_model.file_container_model.batch_insight_model.result
res = tmp.loc[(tmp['fileName'] == 'E55H060478_LE_JLADANSUKAHATI_CBX_PL_BXP_6_telog.log'), :].reset_index(drop=True).loc[0, :]
dict(res)